# Model 1 Environment (without RUL in state)

In [ ]:
import numpy as np
from scipy.stats import gamma, norm, expon
from collections import defaultdict

class SteelProductionLine:
    def __init__(self):

          # First workstation
          self.n = 20
          self.h1 = 800
          self.h2 = 200
          self.ci = 50
          self.cf = 3000
          self.cu = 100
          self.cr = 230
          self.cs = 0.01

          # Hammer degradation
          self.alpha_T1 = 1
          self.beta_T1 = 0.1
          self.alpha_T2 = 0.5
          self.beta_T2 = 0.12
          self.hammer_degradation = np.zeros(self.n)
          self.total_replaced_hammers = 0
          self.last_num_defective_hammers = 0

          # Production
          self.c1 = 1.0
          self.c2 = 0.8
          self.current_Pt = self.c1 * max(0, norm.rvs(loc=1.0, scale=0.1)) * 20
          self.production_pace = 'T1'

          # Maintenance parameters
          self.Tp = 4
          self.Tu = 1
          self.Ta_mean = 10

          self.last_pm_time = 0
          self.last_cm_time = 0
          self.cumulative_cm = 0
          self.first_ws_stopped = False
          self.second_ws_stopped = False

          # Second workstation
          self.demand_mean = 15
          self.demand_std = 1
          self.failure_rate = 1/336
          self.cm_duration_mean_ws2 = 3
          self.pm_duration_ws2 = 2
          self.WS2_age = 0
          self.lifetime = expon.rvs(scale=1/self.failure_rate)
          self.is_down = False
          self.current_rul = self.lifetime
          self.current_dt = norm.rvs(loc=self.demand_mean, scale=self.demand_std)
          self.unmet_demand_during_m = 0

          # Buffer
          self.buffer_capacity = 1000
          self.current_bt = 0

          # Initial state
          self.current_state_Pt, self.current_state_bt, self.current_state_dt = self.get_state(self.current_Pt, self.current_bt, self.current_dt)
          self.time = 0


    def get_production_rate(self):
        rho_t = max(0, norm.rvs(loc=1.0, scale=0.1))
        hammer_contributions = np.zeros(self.n)
        for i in range(self.n):
            if self.hammer_degradation[i] < 0.2:
                hammer_contributions[i] = 1.0
            elif self.hammer_degradation[i] < 0.3:
                hammer_contributions[i] = 0.8
            elif self.hammer_degradation[i] < 0.4:
                hammer_contributions[i] = 0.6
            else:
                hammer_contributions[i] = 0.0
        if self.production_pace == 'T1':
            Pt = self.c1 * rho_t * np.sum(hammer_contributions)
        else:
            Pt = self.c2 * rho_t * np.sum(hammer_contributions)
        return Pt


    def get_demand(self):
        #Generating demand for the second workstation
        if self.second_ws_stopped:
            return 0
        else:
            return norm.rvs(loc=self.demand_mean, scale=self.demand_std)

    def get_rul(self):
        rul = self.lifetime - self.WS2_age
        return rul

    def get_buffer_level(self, Pt, bt, dt):
        new_bt = bt + Pt - dt
        return max(0, min(new_bt, self.buffer_capacity))

    def get_unmet_demand(self, Pt, bt, dt):
        if dt > Pt + bt:
          return dt - Pt - bt
        else:
          return 0

    def get_production_pace(self, bt):
        if bt >= self.h1:
            self.production_pace = 'T2'
        elif bt <= self.h2:
            self.production_pace = 'T1'
        return self.production_pace

    def get_state(self, Pt, bt, dt):
          # State aggregation for pt
          if Pt >= 0.75 * self.c1 * self.n:
              Pt_state = 0
          elif Pt >= 0.5 * self.c1 * self.n:
              Pt_state = 1
          elif Pt >= 0.25 * self.c1 * self.n:
              Pt_state = 2
          else:
              Pt_state = 3

          # State aggregation for bt
          if bt >= 800:
              bt_state = 4
          elif bt >= 600:
              bt_state = 3
          elif bt >= 400:
              bt_state = 2
          elif bt >= 200:
              bt_state = 1
          else:
              bt_state = 0

          # State aggregation for dt (demand)
          if dt > 0:
              dt_state = 1
          else:
              dt_state = 0


          return Pt_state, bt_state, dt_state

    def update_hammer_degradation(self):
      if self.production_pace == 'T1':
          alpha = self.alpha_T1
          beta = self.beta_T1
      else:
          alpha = self.alpha_T2
          beta = self.beta_T2

      for i in range(self.n):
          degradation_increment = gamma.rvs(a=alpha, scale=beta)
          self.hammer_degradation[i] += degradation_increment
          if self.hammer_degradation[i] > 1.0:
              self.hammer_degradation[i] = 1.0

    def pm_perform_ws1_action1(self):
        self.last_num_defective_hammers = np.sum(self.hammer_degradation >= 0.2)
        if self.last_num_defective_hammers <= 3:
            replacement_cost = self.cr * 3
        else:
            replacement_cost = self.cr * self.last_num_defective_hammers
        maintenance_duration = self.Tp + self.last_num_defective_hammers * self.Tu
        self.hammer_degradation[self.hammer_degradation >= 0.2] = 0.0
        self.last_pm_time = self.time
        return maintenance_duration, replacement_cost


    def pm_perform_ws1(self):
        self.last_num_defective_hammers = np.sum(self.hammer_degradation >= 0.2)
        replacement_cost = self.cr * self.last_num_defective_hammers
        maintenance_duration = self.Tp + self.last_num_defective_hammers * self.Tu
        self.hammer_degradation[self.hammer_degradation >= 0.2] = 0.0
        self.last_pm_time = self.time
        return maintenance_duration, replacement_cost


    def cm_perform_ws1(self):
        self.last_num_defective_hammers = np.sum(self.hammer_degradation >= 0.2)
        replacement_cost = self.cr * self.last_num_defective_hammers
        maintenance_duration = self.Tp + self.last_num_defective_hammers * self.Tu + expon.rvs(scale=self.Ta_mean)
        self.hammer_degradation[self.hammer_degradation >= 0.2] = 0.0
        self.last_cm_time = self.time
        self.cumulative_cm += 1
        return maintenance_duration, replacement_cost


    def cm_perform_ws2(self):
        cm_duration = expon.rvs(scale=self.cm_duration_mean_ws2)
        self.lifetime = expon.rvs(scale=1/self.failure_rate)
        self.WS2_age = 0
        self.current_rul = self.lifetime
        return cm_duration

    def reset(self):
        self.hammer_degradation = np.zeros(self.n)
        self.production_pace = 'T1'
        self.time = 0
        self.last_pm_time = 0
        self.last_cm_time = 0
        self.total_replaced_hammers = 0
        self.unmet_demand_during_m = 0
        self.current_Pt = self.c1 * max(0, norm.rvs(loc=1.0, scale=0.1)) * 20
        self.current_dt = norm.rvs(loc=self.demand_mean, scale=self.demand_std)
        self.lifetime = expon.rvs(scale=1/self.failure_rate)
        self.current_rul = self.lifetime
        self.current_state_Pt, self.current_state_bt, self.current_state_dt = self.get_state(self.current_Pt, self.current_bt, self.current_dt)

        return self.current_state_Pt, self.current_state_bt, self.current_state_dt


    def step(self, action):
        reward = 0
        done = False

        self.first_ws_stopped = False
        self.second_ws_stopped = False
        time_before_m = self.time
        Pt_initial = self.current_Pt
        dt_initial = self.current_dt
        bt_initial = self.current_bt
        lifetime_initial = self.lifetime
        rul_initial = self.current_rul
        Pt_state_initial = self.current_state_Pt
        bt_state_initial = self.current_state_bt
        dt_state_initial = self.current_state_dt
        age_before_m = self.WS2_age
        hammer_degradation_initial = self.hammer_degradation.copy()
        production_pace_initial = self.production_pace

        ws2_maintenance_time = 0
        ws1_maintenance_time = 0


        if self.current_state_Pt == 3:
            cm_duration_ws1, replacement_cost = self.cm_perform_ws1()
            reward -= self.cf + self.ci + replacement_cost
            self.current_Pt = 0
            cm_duration_ws2 = 0
            if self.current_bt == 0:
                  self.current_dt = 0
                  self.second_ws_stopped = True
                  if self.current_rul <= 0:
                    cm_duration_ws2 = self.cm_perform_ws2()
            self.WS2_age += max(cm_duration_ws1, cm_duration_ws2)


            self.first_ws_stopped = True
            Pt_during_m = self.current_Pt
            bt_during_m = self.current_bt
            dt_during_m = self.current_dt

        elif self.time > 0 and self.current_bt <= 0:
            maintenance_duration, replacement_cost = self.pm_perform_ws1()
            reward -= self.ci + replacement_cost
            self.current_Pt = 0
            cm_duration_ws2 = 0
            if self.current_rul <= 0:
                cm_duration_ws2 = self.cm_perform_ws2()

            self.WS2_age += max(maintenance_duration, cm_duration_ws2)
            self.current_dt = 0
            self.second_ws_stopped = True
            self.first_ws_stopped = True
            Pt_during_m = self.current_Pt
            bt_during_m = self.current_bt
            dt_during_m = self.current_dt

        elif self.time > 0 and self.current_bt > self.buffer_capacity:
            maintenance_duration, replacement_cost = self.pm_perform_ws1()
            reward -= self.ci + replacement_cost
            self.current_Pt = 0
            cm_duration_ws2 = 0
            self.WS2_age += max(cm_duration_ws2, maintenance_duration)

            self.first_ws_stopped = True
            Pt_during_m = self.current_Pt
            bt_during_m = self.current_bt
            dt_during_m = self.current_dt

        elif self.current_rul <= 0:
            cm_duration = self.cm_perform_ws2()
            ws2_maintenance_time = cm_duration
            maintenance_duration, replacement_cost = self.pm_perform_ws1()
            reward -= self.ci + replacement_cost
            ws1_maintenance_time = maintenance_duration

            self.current_dt = 0
            self.current_Pt = 0
            max_maintenance_time = max(ws1_maintenance_time, ws2_maintenance_time)
            self.WS2_age += max_maintenance_time

            self.first_ws_stopped = True
            self.second_ws_stopped = True
            Pt_during_m = self.current_Pt
            bt_during_m = self.current_bt
            dt_during_m = self.current_dt

        elif self.time > 0 and int(self.time) % 24 == 0:
            if self.current_rul <= 0:
                cm_duration = self.cm_perform_ws2()
                ws2_maintenance_time = cm_duration
            else:

                ws2_maintenance_time = self.pm_duration_ws2

            maintenance_duration, replacement_cost = self.pm_perform_ws1()
            reward -= self.ci + replacement_cost
            ws1_maintenance_time = maintenance_duration

            self.current_dt = 0
            self.current_Pt = 0
            max_maintenance_time = max(ws1_maintenance_time, ws2_maintenance_time)
            self.WS2_age += max_maintenance_time

            self.first_ws_stopped = True
            self.second_ws_stopped = True
            Pt_during_m = self.current_Pt
            bt_during_m = self.current_bt
            dt_during_m = self.current_dt

        elif action == 1:
            last_num_defective_hammers = np.sum(self.hammer_degradation >= 0.2)
            maintenance_duration, replacement_cost = self.pm_perform_ws1_action1()
            reward -= self.ci + replacement_cost
            self.current_Pt = 0
            cm_duration_ws2 = 0
            if self.time > 0 and self.current_bt <= 0:
                  self.current_dt = 0
                  self.second_ws_stopped = True
                  if self.current_rul <= 0:
                      cm_duration_ws2 = self.cm_perform_ws2()
            self.WS2_age += max(cm_duration_ws2, maintenance_duration)

            self.first_ws_stopped = True
            Pt_during_m = self.current_Pt
            bt_during_m = self.current_bt
            dt_during_m = self.current_dt


        hammer_degradation_after_maintenance = self.hammer_degradation.copy()
        time_after_m = self.time
        age_after_m = self.WS2_age


        self.update_hammer_degradation()
        self.time += 1
        self.WS2_age += 1
        self.current_Pt = self.get_production_rate()
        self.current_dt = self.get_demand()
        self.current_bt = self.get_buffer_level(self.current_Pt, self.current_bt, self.current_dt)
        self.current_rul = self.get_rul()
        self.current_state_Pt, self.current_state_bt, self.current_state_dt = self.get_state(self.current_Pt, self.current_bt, self.current_dt)


        unmet_demand = self.get_unmet_demand(self.current_Pt, self.current_bt, self.current_dt)
        self.production_pace = self.get_production_pace(self.current_bt)
        reward -= self.cu * unmet_demand + self.cs * self.current_bt

        done = self.time >= 1000

        '''print(f"""
*** Inspection and Maintenance Times:
    Time Before Maintenance(valid if it's in a maintenance state): {time_before_m}
    Time After Maintenance(valid if it's in a maintenance state): {time_after_m}
    Inspection Time At the End: {self.time}
    Action: {action}
    Reward: {reward}


*** Initial Values:
    Production Rate (Pt): {Pt_initial}
    Demand (dt): {dt_initial}
    Buffer level (bt): {bt_initial}
    State -> Pt_state: {Pt_state_initial}, bt_state: {bt_state_initial}
    Hammer Degradation (Initial): {hammer_degradation_initial}
    production pace: {production_pace_initial}
""")

        if self.first_ws_stopped:
              print(f"""
*** During Maintenance:
    Production Rate (Pt): {Pt_during_m}
    Demand (dt): {dt_during_m}
    Buffer level (bt): {bt_during_m}
""")

        print(f"""
*** After 1 step:
    Production Rate (Pt): {self.current_Pt}
    Demand (dt): {self.current_dt}
    Buffer level (bt): {self.current_bt}
    State -> Pt_state: {self.current_state_Pt}, bt_state: {self.current_state_bt}, dt_state: {self.current_state_dt}
    Total Replaced Hammers: {self.last_num_defective_hammers}
    Hammer Degradation (After Maintenance): {hammer_degradation_after_maintenance}
    Hammer Degradation (After 1 step): {self.hammer_degradation}
    production pace: {self.production_pace}
    Unmet Demand: {unmet_demand}
""")

        print(f"""
*** next state: (Pt, bt, dt):{self.current_state_Pt, self.current_state_bt, self.current_state_dt}

""")
        print("-" * 150)'''

        return self.current_state_Pt, self.current_state_bt, self.current_state_dt, reward, done



# Q-Learning for Model 1

In [ ]:
num_episodes = 10000
max_steps = 500
learning_rate = 0.01
discount_factor = 0.99
epsilon = 0.7
epsilon_decay = 0.9995
min_epsilon = 0.01

env = SteelProductionLine()

num_pt_states = 4
num_bt_states = 5
num_dt_states = 2
num_states = num_pt_states * num_bt_states * num_dt_states
num_actions = 2


Q = np.zeros((num_states, num_actions))


def state_to_index(state):
    Pt_state, bt_state, dt_state = state
    index = Pt_state * (num_bt_states * num_dt_states) + bt_state * (num_dt_states) + dt_state
    return int(index)


state_counts = defaultdict(int)


reward_list = []
q_value_changes = []


for episode in range(num_episodes):
    state = env.reset()
    state_idx = state_to_index(state)
    total_reward = 0
    episode_q_value_change = 0
    num_step = 0
    for step in range(max_steps):
        if np.random.rand() < epsilon:
            action = np.random.choice(num_actions)
        else:
            action = np.argmax(Q[state_idx])

        next_state_pt, next_state_bt, next_state_dt, reward, done = env.step(action)
        next_state = (next_state_pt, next_state_bt, next_state_dt)
        next_state_idx = state_to_index(next_state)

        old_q_value = Q[state_idx, action]
        Q[state_idx, action] += learning_rate * (
            reward + discount_factor * np.max(Q[next_state_idx]) - old_q_value)

        episode_q_value_change += abs(Q[state_idx, action] - old_q_value)
        state_counts[state_idx] += 1

        state = next_state
        state_idx = next_state_idx
        total_reward += reward
        num_step += 1
        if done:
            break

    epsilon = max(min_epsilon, epsilon * epsilon_decay)
    reward_list.append(total_reward)
    q_value_changes.append(episode_q_value_change)


    if (episode + 1) % 100 == 0:
        print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Epsilon: {epsilon}, Steps: {num_step}")


print("Q-Table:")
print(Q)


print("State distribution:")
for state_idx, count in state_counts.items():
    print(f"State {state_idx}: {count} occurrences")



## Plots and Q-table

In [ ]:
# Plot total reward over episodes
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

plt.plot(reward_list)
plt.title('Total Reward Over Episodes')
plt.xlabel('Episode')
plt.ylabel('Total Reward')

plt.show()

In [ ]:
# Moving Average Reward Over Episodes

window_size = 100
moving_average = np.convolve(reward_list, np.ones(window_size), 'valid') / window_size
plt.figure(figsize=(12, 6))
plt.plot(moving_average)
plt.title('Moving Average Reward Over Episodes')
plt.xlabel('Episode')
plt.ylabel('Moving Average Reward')
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

num_pt_states = 4  # P=0,1,2,3
num_bt_states = 5  # B=0,1,2,3,4
num_dt_states = 2  # D=0,1

table_data = []
for bt in range(num_bt_states):
    for dt in range(num_dt_states):
        row_header = [f"B={bt}", f"D={dt}"]
        action_values = []
        for Pt in range(num_pt_states):
            for action in [0, 1]:
                state_idx = Pt * (num_bt_states * num_dt_states) + bt * num_dt_states + dt
                action_values.append(Q[state_idx][action])
        table_data.append(row_header + action_values)

columns = ['B', 'D']
for Pt in range(num_pt_states):
    columns.extend([f'P={Pt} (A0)', f'P={Pt} (A1)'])

df = pd.DataFrame(table_data, columns=columns)

plt.figure(figsize=(16, 10))
sns.set(font_scale=0.8)
sns.set_style("whitegrid")

heatmap = sns.heatmap(
    df.set_index(['B', 'D']),
    annot=True,
    fmt=".2f",
    cmap="YlGnBu",
    linewidths=0.5,
    cbar_kws={'label': 'Q-Value'},
    annot_kws={'size': 8}
)

plt.title("Q-Value Heatmap - Action-Value Pairs per Production State", pad=20)
plt.xlabel("Production State (P) with Action Pairs", labelpad=15)
plt.ylabel("Buffer (B) & Demand (D) States", labelpad=15)
heatmap.xaxis.tick_top()
heatmap.xaxis.set_label_position('top')

plt.tight_layout()
plt.show()

## Evaluation of the trained Q-Learning agent

In [ ]:
# -----------------------------------------------
# Evaluation of the trained Q-Learning agent
#    with greedy action selection
# -----------------------------------------------

eval_episodes = 500
eval_epsilon  = 0
eval_rewards  = []

def eps_greedy_policy(state_idx, Q, eps, act_space):
    if np.random.rand() < eps:
        return np.random.choice(act_space)
    else:
        return act_space[np.argmax(Q[state_idx])]

act_space = np.arange(num_actions)

for _ in range(eval_episodes):
    state     = env.reset()
    state_idx = state_to_index(state)
    ep_reward = 0

    while True:
        action = eps_greedy_policy(state_idx, Q, eval_epsilon, act_space)
        next_pt, next_bt, next_dt, reward, done = env.step(action)

        state     = (next_pt, next_bt, next_dt)
        state_idx = state_to_index(state)

        ep_reward += reward
        if done:
            break

    eval_rewards.append(ep_reward)

print(f"ε = {eval_epsilon}")
print(f"Average reward from the QL trained agent (thesis) during {eval_episodes} episodes: {np.mean(eval_rewards):.2f}")
print(f"Max episode reward: {np.max(eval_rewards):.2f}")
print(f"Min episode reward:  {np.min(eval_rewards):.2f}")
